In [ ]:
import os
from datetime import datetime, timedelta
import glob
from multiprocessing import Pool
import pandas as pd
import seaborn as sns
from cartopy.crs import NorthPolarStereo
from cartopy.feature import LAND, COASTLINE
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import numpy as np
from cmocean import cm

DAY_SECONDS = 24 * 60 * 60

In [ ]:
def decorate_map(ax, map_extent, crs, title):
    ax.add_feature(LAND)
    ax.add_feature(COASTLINE)
    ax.set_extent(map_extent, crs=crs)
    ax.set_title(title)
    ax.plot(0, 0, 'bo')
    ax.text(30000, -30000, 'North \nPole')

srs_dst = NorthPolarStereo(central_longitude=-45, true_scale_latitude=60)
map_extent = [-2300000, 300000, -1000000, 2100000]
titles = ['RGPS', 'BBM']

In [ ]:
odir = '../tuning_paper_figures'
idir = '../music_matrix/cfg04_m20'
pfiles = [f'{idir}/mat{best_param_i:02}_pairs.npz' for best_param_i in range(10)]
sources = ['neXtSIM'] * len(pfiles)

pfiles += ['../rgps_csv/w07_may_pairs.npz']
sources += ['RGPS']

In [ ]:
names = ['div', 'she']
cmaps = [cm.balance, 'plasma_r']
full_names = [
    'Divergence',
    'Shear',
]
vmax = 0.1
vmins = [-vmax, 0]

force = True

for pfile, src in zip(pfiles, sources):
    dfile = pfile.replace('_pairs.npz', '_defor.npz')
    pairs = np.load(pfile, allow_pickle=True)['pairs']
    defor = np.load(dfile, allow_pickle=True)['defor']

    date0 = datetime(2007,1,1)
    period_len = timedelta(4)
    time_step = 3

    for day in range(0, 48, time_step):
        date1 = date0 + timedelta(day)
        bfile = pfile.rstrip('_pairs.npz')
        for i, (name, cmap, full_name, vmin) in enumerate(zip(names, cmaps, full_names, vmins)):
            ofile = f'{odir}/{bfile}_{name}_{date1.strftime("%Y%m%d")}.png'
            if os.path.exists(ofile) and not force:
                continue

            date_str = date1.strftime('%Y-%m-%d')
            print(full_name, date1)
            fig, axs = plt.subplots(1,1,figsize=(10,10), subplot_kw={'projection': srs_dst})
            for p, d in zip(pairs, defor):
                if p is None or d is None:
                    continue
                if date1 <= p.d1 < (date1 + period_len):
                    e = [d.e1, d.e2][i] * DAY_SECONDS
                    trp = axs.tripcolor(p.x1, p.y1, e, triangles=p.t, vmin=vmin, vmax=0.1, cmap=cmap, mask=~p.g)
            decorate_map(axs, map_extent, srs_dst, src)
            plt.tight_layout()

            bgax = inset_axes(axs, '95%', '10%', loc='lower center')
            bgax.set_facecolor((1,1,1,0.9))
            bgax.spines['top'].set_visible(False)
            bgax.spines['right'].set_visible(False)
            bgax.spines['bottom'].set_visible(False)
            bgax.spines['left'].set_visible(False)
            bgax.tick_params(colors=(0,0,0,0), )
            cbar_ax = inset_axes(bgax, '95%', '15%', loc='upper center')
            cbar = fig.colorbar(trp, cax=cbar_ax, orientation='horizontal')
            cbar.set_label(f'{full_name}, {date_str}, 1/day', fontsize=14)

            print(ofile)
            plt.savefig(ofile, dpi=150, bbox_inches='tight', pad_inches=0)
            plt.close()

In [ ]:
idir = './music_matrix/cfg04_m20'

ndiv = []
nshe = []

for i in range(10):
    ndiv.append(sorted(glob.glob(f'{idir}/mat0{i}_div_*.png')))
    nshe.append(sorted(glob.glob(f'{idir}/mat0{i}_she_*.png')))

rdiv = sorted(glob.glob('../rgps_csv/w07_may_div*.png'))
rshe = sorted(glob.glob('../rgps_csv/w07_may_she*.png'))

len(ndiv[0]), len(nshe[0]), len(rdiv), len(rshe)

In [ ]:
for i in range(10):
    ndivi = ndiv[i]
    nshei = nshe[i]
    for rd, rs, nd, ns in zip(rdiv, rshe, ndivi, nshei):
        fig, axs = plt.subplots(1, 4, figsize=(20,6))
        for j, f in enumerate([rd, nd, rs, ns]):
            a = plt.imread(f)
            axs[j].imshow(a[100:-50, 20:-10])
        for ax in axs.flat:
            ax.axis('off')
        date = f.split('.')[-2].split('_')[-1]
        axs[0].text(30, 750, date, fontsize=16, rotation=90)
        axs[0].text(250, 50, 'RGPS', fontsize=14)
        axs[1].text(250, 50, 'neXtSIM', fontsize=14)
        axs[2].text(250, 50, 'RGPS', fontsize=14)
        axs[3].text(250, 50, 'neXtSIM', fontsize=14)

        plt.tight_layout()
        ofile = f'{idir}/row_{i}_{date}.png'
        plt.savefig(ofile, dpi=150, bbox_inches='tight', pad_inches=0)
        plt.close()
        print(ofile)